<h1 align=center>SVC</h1>

---

In [46]:
import os #type: ignore
import json #type: ignore
import numpy as np #type: ignore
import pandas as pd #type: ignore
from sklearn.svm import SVC #type: ignore
from sklearn.model_selection import train_test_split #type: ignore
from sklearn.metrics import classification_report, accuracy_score #type: ignore


In [47]:
# Load dataset
def load_dataset(folder):
    """
    Loads the dataset and flattens input-output image pairs.
    """
    X, y = [], []
    for fname in os.listdir(folder):
        if fname.endswith(".json"):
            # Load input-output data
            with open(os.path.join(folder, fname), "r") as f:
                data = json.load(f)

            for sample in data:
                inp_img = np.array(sample["input"]).flatten()
                out_img = np.array(sample["output"]).flatten()
                X.append(np.concatenate([inp_img, out_img]))

                # Load solution file
                soln_fname = fname.replace(".json", "_soln.txt")
                with open(os.path.join(folder, soln_fname), "r") as f:
                    se_indices = [int(line.split("SE")[1]) - 1 for line in f][:4]
                y.append(se_indices)

    # Flatten labels to make it one-vs-one classification
    return np.array(X), np.array(y)

# Prepare data
dataset_folder = "CatA_Simple"
X, y = load_dataset(dataset_folder)


In [48]:
X_df = pd.DataFrame(X)
y_df = pd.DataFrame(y)

In [49]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((320, 450), (80, 450), (320, 4), (80, 4))

In [50]:
# Train an SVC model for each SE position
models = []
for i in range(y.shape[1]):  # One model for each SE in the sequence
    svc = SVC(kernel="linear", random_state=42)
    svc.fit(X_train, y_train[:, i])
    models.append(svc)

# Evaluate the models
y_pred = np.zeros_like(y_test)
for i, svc in enumerate(models):
    y_pred[:, i] = svc.predict(X_test)

# Accuracy and classification report
accuracy = accuracy_score(y_test.flatten(), y_pred.flatten())
print(f"Overall Accuracy: {accuracy*100:.2f}%")
print(classification_report(y_test.flatten(), y_pred.flatten()))


Overall Accuracy: 22.50%
              precision    recall  f1-score   support

           0       0.14      0.18      0.16        44
           1       0.21      0.16      0.18        43
           2       0.19      0.13      0.15        31
           3       0.15      0.18      0.17        38
           4       0.28      0.32      0.30        34
           5       0.22      0.18      0.20        39
           6       0.33      0.35      0.34        48
           7       0.29      0.26      0.27        43

    accuracy                           0.23       320
   macro avg       0.23      0.22      0.22       320
weighted avg       0.23      0.23      0.22       320

